In [1]:
import socket
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Load the IMDb dataset
dataset = load_dataset('imdb')

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2)

# Preprocess the dataset
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',  # Ensure padding to max_length
        truncation=True,
        max_length=128  # Set this according to model constraints
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

trainer.train()
print("Model training complete.")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Exam\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Exam\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,0.386500,0.367773
2,0.328100,0.353482
3,0.282100,0.349449


Model training complete.


In [3]:
predictions

tensor([1, 0])

In [4]:
import socket
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2)

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the correct device

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 10300))  # Bind to all available interfaces and port 10300
server_socket.listen(1)

print("Server is listening for connections...")
conn, addr = server_socket.accept()
print(f"Connected to {addr}")

# Receive input data from the client
data = conn.recv(4096)
inputs = pickle.loads(data)

# Move inputs to the same device as the model
for key in inputs.keys():
    inputs[key] = inputs[key].to(device)

# Make predictions using the model
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)  # Get the predicted class (0 or 1)

# Serialize the predictions and send them back to the client
predictions_data = pickle.dumps(predictions.cpu())  # Move to CPU before serializing
conn.sendall(predictions_data)

conn.close()
print("Connection closed.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Server is listening for connections...
Connected to ('172.16.19.7', 57464)


UnpicklingError: pickle data was truncated

In [ ]:
import socket
import pickle

# Function to receive data in chunks from the client
def receive_data(conn):
    buffer_size = 4096  # Adjust buffer size as needed
    data = b""  # Use a bytes object to store the received data
    
    # Keep receiving data in chunks until there is no more data
    while True:
        part = conn.recv(buffer_size)
        if not part:
            break  # Stop receiving when no more data is sent
        data += part  # Append each chunk to the data

    return pickle.loads(data)  # Unpickle the complete data after receiving all parts

# Set up the server
def server_program():
    host = '0.0.0.0'  # Localhost
    port = 10300        # Arbitrary non-privileged port
    
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))  # Bind the server to the host and port
    server_socket.listen(1)           # Listen for incoming connections

    print("Server is listening for connections...")
    
    # Accept a connection from the client
    conn, address = server_socket.accept()
    print(f"Connected to {address}")
    
    try:
        # Receive and process the data from the client
        inputs = receive_data(conn)
        print("Received data:", inputs)
        
        # Your code for handling the received data goes here
        
    except Exception as e:
        print(f"Error receiving data: {e}")
    
    finally:
        conn.close()  # Close the connection
        print("Connection closed")

# Start the server
if __name__ == "__main__":
    server_program()


In [11]:
import socket
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2)

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the correct device

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 10300))  # Bind to all available interfaces and port 10300
server_socket.listen(1)

print("Server is listening for connections...")
conn, addr = server_socket.accept()
print(f"Connected to {addr}")

def receive_data(conn):
    # First receive the length of the data
    data_length = int.from_bytes(conn.recv(4), 'big')  # Receive 4 bytes (for data size)
    data = b''
    
    # Keep receiving data until the full length is received
    while len(data) < data_length:
        packet = conn.recv(4096)
        if not packet:
            break
        data += packet
    
    return pickle.loads(data)  # Unpickle the complete received data

try:
    # Receive input data from the client
    inputs = receive_data(conn)

    # Move inputs to the same device as the model
    for key in inputs.keys():
        inputs[key] = inputs[key].to(device)

    # Make predictions using the model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)  # Get the predicted class (0 or 1)

    # Serialize the predictions and send them back to the client
    predictions_data = pickle.dumps(predictions.cpu())  # Move to CPU before serializing
    conn.sendall(len(predictions_data).to_bytes(4, 'big'))  # Send the length of the data first
    conn.sendall(predictions_data)  # Send the actual predictions

finally:
    conn.close()
    print("Connection closed.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Server is listening for connections...
Connected to ('172.16.19.59', 52549)
Connection closed.


UnpicklingError: invalid load key, '8'.

In [4]:
from datasets import load_dataset
import pandas as pd

# Load the IMDb dataset using Hugging Face's datasets library
dataset = load_dataset('imdb')

# Convert the train and test splits to pandas DataFrames
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

# Display the first few rows of the train DataFrame
print(train_df.head())



                                                text  label
0  I rented I AM CURIOUS-YELLOW from my video sto...      0
1  "I Am Curious: Yellow" is a risible and preten...      0
2  If only to avoid making this type of film in t...      0
3  This film was probably inspired by Godard's Ma...      0
4  Oh, brother...after hearing about this ridicul...      0


In [6]:
test_df['text'][0]

'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they have

## trained one final

In [12]:
import socket
import torch
import pickle
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the IMDb dataset
print("Loading IMDb dataset...")
dataset = load_dataset('imdb')

# Load the tokenizer and model
print("Loading TinyBERT model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
model = AutoModelForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2)

# Preprocess the dataset for model input
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',  # Ensure padding to max_length
        truncation=True,
        max_length=128  # Truncate longer reviews
    )

print("Tokenizing the dataset...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Split dataset into train and test sets
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the correct device

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',        # Output directory
    evaluation_strategy="epoch",   # Evaluate after every epoch
    learning_rate=2e-5,            # Learning rate
    per_device_train_batch_size=16,# Training batch size
    per_device_eval_batch_size=16, # Evaluation batch size
    num_train_epochs=3,            # Number of training epochs
    weight_decay=0.01,             # Weight decay
)

# Trainer setup
print("Initializing the trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
print("Training the model on IMDb dataset...")
trainer.train()

# Save the model after training
print("Saving the trained model...")
model.save_pretrained('./trained_model')
tokenizer.save_pretrained('./trained_model')

# Now that the model is trained, set up the server for inference requests

# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 10300))  # Bind to all available interfaces and port 10300
server_socket.listen(1)

print("Server is listening for connections...")
conn, addr = server_socket.accept()
print(f"Connected to {addr}")

def receive_data(conn):
    # First receive the length of the data
    data_length = int.from_bytes(conn.recv(4), 'big')  # Receive 4 bytes (for data size)
    data = b''
    
    # Keep receiving data until the full length is received
    while len(data) < data_length:
        packet = conn.recv(4096)
        if not packet:
            break
        data += packet
    
    return pickle.loads(data)  # Unpickle the complete received data

try:
    # Receive input data from the client
    inputs = receive_data(conn)

    # Move inputs to the same device as the model
    for key in inputs.keys():
        inputs[key] = inputs[key].to(device)

    # Make predictions using the model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)  # Get the predicted class (0 or 1)

    # Serialize the predictions and send them back to the client
    predictions_data = pickle.dumps(predictions.cpu())  # Move to CPU before serializing
    conn.sendall(len(predictions_data).to_bytes(4, 'big'))  # Send the length of the data first
    conn.sendall(predictions_data)  # Send the actual predictions

finally:
    conn.close()
    print("Connection closed.")


Loading IMDb dataset...
Loading TinyBERT model and tokenizer...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Exam\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Tokenizing the dataset...
Initializing the trainer...
Training the model on IMDb dataset...


Epoch,Training Loss,Validation Loss
1,0.385300,0.367260
2,0.328200,0.355444
3,0.281000,0.349609


Saving the trained model...
Server is listening for connections...
Connected to ('172.16.19.59', 52728)
Connection closed.


### run this

In [24]:
# Server setup
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 10300))  # Bind to all available interfaces and port 10300
server_socket.listen(1)

print("Server is listening for connections...")
conn, addr = server_socket.accept()
print(f"Connected to {addr}")

def receive_data(conn):
    # First receive the length of the data
    data_length = int.from_bytes(conn.recv(4), 'big')  # Receive 4 bytes (for data size)
    data = b''
    
    # Keep receiving data until the full length is received
    while len(data) < data_length:
        packet = conn.recv(4096)
        if not packet:
            break
        data += packet
    
    return pickle.loads(data)  # Unpickle the complete received data

try:
    # Receive input data from the client
    inputs = receive_data(conn)

    # Move inputs to the same device as the model
    for key in inputs.keys():
        inputs[key] = inputs[key].to(device)

    # Make predictions using the model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)  # Get the predicted class (0 or 1)

    # Serialize the predictions and send them back to the client
    predictions_data = pickle.dumps(predictions.cpu())  # Move to CPU before serializing
    conn.sendall(len(predictions_data).to_bytes(4, 'big'))  # Send the length of the data first
    conn.sendall(predictions_data)  # Send the actual predictions

finally:
    conn.close()
    print("Connection closed.")


Server is listening for connections...
Connected to ('172.16.19.59', 52857)
Connection closed.


In [23]:
predictions

tensor([1, 0], device='cuda:0')